In [14]:
import pickle
import numpy as np

data_dir = '/home/sangwon/Projects/ai_coach/train_ma_dnn/data'
file_name = 'sc2_2s3z_100.pkl'

with open(f'{data_dir}/{file_name}', 'rb') as f:
  expert_trajs = pickle.load(f)

wons = expert_trajs["wons"]
lengths = expert_trajs["lengths"]

print(np.mean(wons), np.std(wons))
print(np.mean(lengths), np.std(lengths))
rewards = expert_trajs["rewards"]
np_rewards = np.array(rewards)
np_sum_rewards = np.sum(np_rewards, axis=2).squeeze()
print(np_sum_rewards.shape)
print(np.mean(np_sum_rewards[:, 0]), np.std(np_sum_rewards[:, 0]))





0.97 0.1705872210923198
120.0 0.0
(100, 5)
19.930613 0.97076434


In [2]:
n_episodes = len(expert_trajs["latents"])
n_agents = len(expert_trajs["latents"][0])
# print(n_episodes, n_agents)

n_latents = 4

latent_stats = np.zeros((n_agents, n_episodes, n_latents))

for i_e, episode in enumerate(expert_trajs["latents"]):
  for i_a, latents in enumerate(episode):
    for latent in latents:
      latent_stats[i_a, i_e, latent] += 1

print("average latent per episode:")
print(np.mean(latent_stats, axis=1))

print(np.std(latent_stats, axis=1))



average latent per episode:
[[74.52 17.19 21.89  6.4 ]
 [76.57  8.74 28.69  6.  ]
 [75.64 16.22 19.6   8.54]
 [67.32 25.94 17.91  8.83]
 [63.86 33.55 11.51 11.08]]
[[16.644807   16.22879848 10.63851023  8.1215762 ]
 [14.05507382  8.40073806 11.57730107  5.81205643]
 [12.11983498  9.68460634 11.01998185 10.54553934]
 [13.64835521 13.48986286 12.36535078  8.20616232]
 [16.90030769 19.38523923  6.48458942  9.39327419]]
